# Tests per agegroup
Data extracted from weekly report of RIVM

In [1]:
import os
import pandas as pd
import tabula
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re

rivm_url = 'https://www.rivm.nl/coronavirus-covid-19/actueel/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland'
response = requests.get(rivm_url)
rivm_pdf_data_loc = './data/pdfs/'

In [2]:
href = []
period = []
name = []
datetime = []
week = []
filename = []
url = []

def columns(l):
    href.append(l)
    period.append(pd.Period(l[-l.count('')+22:-l.count('')+29],"M"))
    name.append(l[-l.count('')+30:-4])
    datetime.append(pd.to_datetime(l[-l.count('')+65:-l.count('')+73], format="%Y%m%d"))
    filename.append(l[-l.count('')+65:])
    url.append('https://www.rivm.nl'+l)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser') 
aa = soup.find_all('a')
for a in aa:
    link = a.get('href')
    if link == None:
        pass
    elif link.endswith('.pdf'):
        columns(link)

pdfs = pd.DataFrame([period, datetime, week, name, filename, href, url]).T
pdfs = pdfs.rename(columns={0:'period',1:'date',2:'week',3:'name',4:'filename',5:'href',6:'url'})
pdfs['week'] = pdfs.date.dt.isocalendar().week

In [48]:
def extract_tabel(pdf,csv,date):
    
    def check_tabel(value):
        for key in checks:
            if checks[key]:
                pass
            elif isinstance(value,str) and key in value:
                checks[key] = True
    
    def create_agegroup_tabel(tabel):
        #print(tabel)
        if checks['Leeftijdsgroep'] and checks['fgelopen']:
            category = 'Leeftijd'
        elif checks['Groep'] and checks['fgelopen'] and checks['Kinderen']:
            category = 'Groep'
        elif checks['Groep'] and checks['Kinderen']:
            category = 'Groep'
        else:
            category = ''
        #print('cat='+category)
        
        if category != '':
            tabel.dropna(inplace=True)
            if tabel.empty:
                #print("df empty")
                pass
            else:
                first_column = tabel.columns[0]
                #first_column = 'Unnamed: 0'
                if len(tabel.columns) > 1:
                    second_column = tabel.columns[1]
                agegroup_tabel = tabel.loc[(tabel[first_column].str.count('-') > 0)]                
                if category == 'Leeftijd':
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.find('+') > 0)])
                    agegroup_tabel.rename(columns={first_column:'Leeftijdsgroep',second_column:date}, inplace=True)
                elif category == 'Groep':
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('fgelopen') > 0)])
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('Groep') > 0)])
                agegroup_tabel = agegroup_tabel.sort_index()
                agegroup_tabel.to_csv(csv, index=False)
                #print('CSV CREATED')
                #print(agegroup_tabel)
                return True
        #else:
            #print("NOTHING")
    
    print('extracting '+pdf)
    #print(pdf)
    found_pages = []
    object = PyPDF2.PdfFileReader(pdf)
    NumPages = object.getNumPages()
    String = "SARS-COV-2.*TESTEN.*AFGENOMEN.*DOOR.*DE.*GGD"
    
    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        Text = PageObj.extractText()
        if re.search(String,Text):
            found_pages.append(i)
    
    if len(found_pages) == 0:
        print("NO CHAPTER FOUND in "+str(pdf))
    else:
        tabels = tabula.read_pdf(input_path=pdf, pages=found_pages, guess=True, stream=False)
        csv_created = None
        for tabel in tabels:
            checks = {
                'Leeftijdsgroep': False,
                'fgelopen': False,
                'Groep': False,
                'Kinderen': False}
            for column in tabel:
                check_tabel(column)
                for i in tabel[column].values:
                    check_tabel(i)
            #print(checks)
            csv_created = create_agegroup_tabel(tabel)
            #print(csv_created)
        if csv_created == None:
            no_tabel_found.append(pdf)
            print('No tabel found in: '+pdf)

In [49]:
# download all pdfs
no_tabel_found = []
for row in pdfs.itertuples():
    pdf = rivm_pdf_data_loc+row.filename
    csv = pdf[:-3]+'csv'
    try:
        os.stat(csv)
        print('skiping '+csv+' exists')
    except:
        #print(pdf)
        try:
            os.stat(path)
        except:
            print('downloading '+row.filename)
            r = requests.get(row.url, stream=True)
            with open(pdf, 'wb') as f:
                f.write(r.content)
        finally:
            extract_tabel(pdf, csv, row.date)
print("DONE")

downloading 20210323_1159.pdf
extracting ./data/pdfs/20210323_1159.pdf


Got stderr: Mar 24, 2021 9:24:54 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 24, 2021 9:24:54 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



No tabel found in: ./data/pdfs/20210323_1159.pdf
downloading 20210316_1159.pdf
extracting ./data/pdfs/20210316_1159.pdf


Got stderr: Mar 24, 2021 9:29:37 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 24, 2021 9:29:37 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



No tabel found in: ./data/pdfs/20210316_1159.pdf
extracting ./data/pdfs/20210309_1259.pdf


...

No tabel found in: ./data/pdfs/20201201_1159.pdf
extracting ./data/pdfs/20201208_1159.pdf


Got stderr: Mar 24, 2021 9:38:16 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 24, 2021 9:38:16 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



No tabel found in: ./data/pdfs/20201208_1159.pdf
extracting ./data/pdfs/20201215_1259.pdf


Got stderr: Mar 24, 2021 9:38:30 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 24, 2021 9:38:30 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



No tabel found in: ./data/pdfs/20201215_1259.pdf
DONE


In [19]:
# TEST
print(no_tabel_found)
pdf = no_tabel_found[0]

csv = pdf[:-3]+'csv'
try:
    os.stat(csv)
    print('Skipping csv exists')
except:
    print(pdf)
    try:
        os.stat(pdf)
    except:
        print('downloading')
        #r = requests.get(row.url, stream=True)
        #with open(path, 'wb') as f:
        #   f.write(r.content)
    finally:
        extract_tabel(pdf, csv, 'xxx')

['./data/pdfs/20200714_1040.pdf', './data/pdfs/20200714_1040.pdf']
./data/pdfs/20200714_1040.pdf
extracting ./data/pdfs/20200714_1040.pdf


Got stderr: Mar 24, 2021 9:04:56 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Mar 24, 2021 9:04:56 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



No tabel found in: ./data/pdfs/20200714_1040.pdf


In [ ]:
# clean up extracted tabels
## values in string as list. Find out what value to use.
##
data = pd.DataFrame()
n = 0
for filename in os.listdir(rivm_pdf_data_loc):
    if '.csv' in filename:
        path = rivm_pdf_data_loc+filename
        df = pd.read_csv(path)
        
        print(df)
        print('---')
        
        if n == 0: 
            data = pd.concat([data,df])
            n+=1
        else:
            data = data.merge(df, on='Leeftijdsgroep', how='outer')
            break
print('===')
print(data)

In [ ]:
# create data df
n = 0
for...
    print('tabel to data')
    if n == 0:
        data = tab
        data = data.rename(columns={data.columns[0]:'Leeftijdsgroep',data.columns[1]:row.date})
        #print(data.columns)
    else:
        tab = tab.rename(columns={tab.columns[0]:'Leeftijdsgroep',tab.columns[1]:row.date})
        data = data.merge(tab, on=data.columns[0], how='outer')
        #print(data.columns)
n+=1
data.to_csv(csv, index=False)
print('done')

In [ ]:
data

## Chapter 7 or 9

## What can be used from the reports

### Chapter 4 agegroups
Reported, Hospitalised, Deceased agegroups

https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.json